<a href="https://colab.research.google.com/github/Charu-Mittal12/Person_tracking_and_detection/blob/main/person_detection_and_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 32.2 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO
import os
from vidgear.gears import CamGear
import dlib
from google.colab.patches import cv2_imshow
import time
import numpy as np

from deep_sort_realtime.deepsort_tracker import DeepSort


In [ ]:


# Load an official or custom model
model = YOLO('yolov8l-pose.pt')  # Load an official Detect model

# Open the saved video
cap = CamGear(source='https://www.youtube.com/watch?v=aulLej6Z6W8',stream_mode =True, logging = True).start()
tracker = DeepSort(nms_max_overlap=1,max_cosine_distance=0.5)


15:55:45 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
15:55:45 ::    CamGear    ::   INFO   :: Verifying Streaming URL using yt-dlp backend. Please wait...
15:55:47 ::    CamGear    ::   INFO   :: [Backend] :: Streaming URL is fully supported. Available Streams are: [best, worst]
15:55:47 ::    CamGear    ::  DEBUG   :: Using `best` resolution for streaming.
15:55:47 ::    CamGear    ::  DEBUG   :: YouTube source ID: `aulLej6Z6W8`, Title: `Matching`, Quality: `best`
15:55:47 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!


In [ ]:
# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec (XVID is a popular choice)
#output_file = 'output_video.avi'  # Output video file name
fps = 30  # Frames per second (adjust if needed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
output_file = '/content/drive/My Drive/output_video1.avi'
# Get the frame size from the underlying stream object
frame_size = (int(cap.stream.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.stream.get(cv2.CAP_PROP_FRAME_HEIGHT)))  # Frame size

out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

In [ ]:
import os

checkpoint_dir = '/content/drive/My Drive/checkpoints/'
os.makedirs(checkpoint_dir, exist_ok=True)

frame_count = 0
checkpoint_interval = 300  # Save every 300 frames

while True:
    frame = cap.read()

    if frame is None:
        break  # Exit loop if no more frames

    # Detect persons using YOLO
    results = model(frame)[0]
    detected_boxes = []
    id = []
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        if class_id == 0:
            detected_boxes.append(([x1, y1, x2, y2], score, class_id))

    # Track objects
    tracks = tracker.update_tracks(detected_boxes, frame=frame)

    # Draw tracking IDs
    for track in tracks:
        track_id = track.track_id
        id.append([track_id, track])
        bbox = track.to_tlbr()
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)
        cv2.putText(frame, f'ID: {track_id}', (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

    # Save a checkpoint frame
    if frame_count % checkpoint_interval == 0:
        checkpoint_file = os.path.join(checkpoint_dir, f'frame_{frame_count}.jpg')
        cv2.imwrite(checkpoint_file, frame)

    # Display the frame using cv2_imshow
    cv2_imshow(frame)
    time.sleep(1 / 30)  # Adjust sleep to match video frame rate if necessary

    frame_count += 1

# Release the video capture object
cap.stop()
out.release()

# Clean up
cv2.destroyAllWindows()


16:04:09 ::    CamGear    ::  DEBUG   :: Terminating processes.
